# 3.2.1 直方圖處理 (Histogram Processing)

**WBS模組編號**: 2.3.1  
**教學目標**: 掌握影像直方圖的計算、分析與處理技術  
**難度等級**: ⭐⭐⭐ (中級)  
**預估學習時間**: 90 分鐘

---

## 📚 學習目標

完成本模組後，你將能夠：

1. ✅ 理解影像直方圖的定義與意義
2. ✅ 使用 `cv2.calcHist()` 計算並繪製直方圖
3. ✅ 應用直方圖均衡化增強影像對比度
4. ✅ 使用 CLAHE 進行自適應直方圖均衡
5. ✅ 掌握灰階拉伸技術改善影像品質

---

## 🔧 環境準備

本模組需要以下套件：
- OpenCV (cv2)
- NumPy
- Matplotlib

In [ ]:
# Import required libraries
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Set matplotlib to display plots inline
%matplotlib inline

# Configure matplotlib for better display
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.family'] = 'DejaVu Sans'

print('✅ Environment setup complete')
print(f'OpenCV version: {cv2.__version__}')
print(f'NumPy version: {np.__version__}')

## 12-1: 直方圖的含意> 定義 : 直方圖反應了一張圖像內所有像素強度從 `0 至 255 的分佈情形`<br>> 特色 : 能掌握圖像的明暗、對比及像素強度分布> Histograms 是直方圖的意思，在攝影領域一般稱為`曝光直方圖`，主要用以分析一張照片的曝光是否正確。應用在電腦視覺則主要利用它來判斷白平衡、處理 thresholding，也可用於物體追蹤（例如 CamShift 演算法即使用彩色直方圖的變化達到跟蹤目的），另外也可用於圖像檢索目的（例如 Bag-of-words 詞袋演算法）、或應用於機器學習運算…等等。因為 Histograms 為我們統計出各像素`強度頻率`的資訊來呈現整張相片的色彩分佈情況，因此我們得以透過此資訊來猜測該圖片的物件及特性。>><img src="../assets/images/basic/calcHist.jpg"  style='width:60%'>### 直方圖均衡化，用於提高影像的質量, 直方圖均衡化是通過拉伸畫素`強度分佈範圍`來增強影像對比度的一種方法.> 所謂直方圖就是對影像的中的這些畫素點的值進行統計，得到一個統一的整體的灰度概念。直方圖的好處就在於可以清晰瞭解影像的整體灰度分佈，這對於後面依據直方圖處理影像來說至關重要。> 在開發影像處理的程式時，我們時常會需要觀察影像像素值的分佈與特性，以便選用適合的演算法、制定門檻值、設計出適合的影像處理流程。><img src="../assets/images/basic/histogram_equalization.jpg"  style='width:90%'>

## 12-3: 實作直方圖

### Gray image

In [ ]:
import numpy as npimport cv2import matplotlib.pyplot as plto = cv2.imread('../assets/images/basic/lenaColor.png', 0)cv2.imshow('original', o)cv_hist256 = cv2.calcHist([o], [0], None, [256], [0,255])cv_hist16 = cv2.calcHist([o], [0], None, [16], [0,255])hist, bins = np.histogram(o.flatten(), 256, [0,256])  # 拉平cdf = hist.cumsum()                                  # 累加 cdf : cumulate distribution functioncdf_normalized = cdf * hist.max()/ cdf.max()         # 標準化print(hist.max(), cdf.max())fig=plt.figure(figsize=(12, 6))plt.subplot(221)  plt.hist(o.flatten(), 256); plt.title('bin=256')       # flatten 將多維陣列轉換為一維陣列的功能 like flattenplt.plot(cdf_normalized, color='r')                 # plot cdfplt.subplot(222)  plt.hist(o.flatten(), 16);  plt.title('bin=16')plt.subplot(223)  plt.plot(cv_hist256, color='y');  plt.title('cv_hist256')plt.subplot(224)  plt.plot(cv_hist16, color='r');  plt.title('cv_hist16')plt.show()cv2.waitKey()cv2.destroyAllWindows()cv2.waitKey(1)

### color image : cv mean

In [ ]:
import cv2import matplotlib.pyplot as plto=cv2.imread('../assets/images/basic/lenaColor.png')cv2.imshow('original', o)cv_histb = cv2.calcHist([o], [0], None, [256], [0,255])   # [0] Bcv_histg = cv2.calcHist([o], [1], None, [256], [0,255])   # [1] Gcv_histr = cv2.calcHist([o], [2], None, [256], [0,255])   # [2] Rmean, std = cv2.meanStdDev(o);    print(f'mean :\n{mean}\n\nstd :\n{std}')fig=plt.figure(figsize=(14, 8))plt.subplot(221)plt.plot(cv_histb, color='b');  plt.title('cv_hist b');  plt.axvline(x=mean[0], color='r')plt.text(mean[0]+2, cv_histb.max()*.95, f'avg.:{mean[0][0]:.2f}', color='r', fontsize=10)        plt.subplot(222)plt.plot(cv_histg, color='g');  plt.title('cv_hist g');  plt.axvline(x=mean[1], color='r')plt.text(mean[1]+2, cv_histg.max()*.95, f'avg.:{mean[1][0]:.2f}', color='r', fontsize=10)        plt.subplot(223)  plt.plot(cv_histr, color='r');  plt.title('cv_hist r');  plt.axvline(x=mean[2], color='r')plt.text(mean[2]+2, cv_histr.max()*.95, f'avg.:{mean[2][0]:.2f}', color='r', fontsize=10)        color = ('b', 'g', 'r')plt.subplot(224);  plt.title('cv_hist rgb')for i, col in enumerate(color):    histr = cv2.calcHist([o], [i], None, [256], [0, 256])    plt.plot(histr, color = col)plt.show()cv2.waitKey()cv2.destroyAllWindows()cv2.waitKey(1)

### Gray Level 直方圖均衡化處理實例 Histogram Equalization (HE)> 定義 : 通過拉伸影像的像素強度分佈範圍來`增強圖像對比度`，適用於`過曝或背光的圖片`<br>> 缺點 : 對處理的數據不加選擇(全局處理)，如此一來`會增加背景雜訊的對比度`並且`降低有用訊號(特別亮或暗)的對比度`經過比較可以發現均衡化後的圖像明亮的部分出現了`過曝`，失去了原本的細節。這是因為原圖整體暗的部份較多，而經過直方圖均衡化後使原本亮的地方更亮，

### gray level Histogram Equalization

In [ ]:
import cv2import numpy as npimport matplotlib.pyplot as plto = cv2.imread('../assets/images/basic/lenaColor.png', 0)equ=cv2.equalizeHist(o)# cv2.imshow('original', o)# cv2.imshow('equ', equ)print(o.size)hist, bins = np.histogram(o.flatten(),256,[0,256])cdf = hist.cumsum()cdf_normalized_o = cdf * hist.max()/ cdf.max()hist, bins = np.histogram(equ.flatten(),256,[0,256])cdf = hist.cumsum()cdf_normalized_eq = cdf * hist.max()/ cdf.max()fig=plt.figure(figsize=(14, 4))plt.subplot(121)plt.hist(o.flatten(), 256)plt.plot(cdf_normalized_o, color='r')plt.subplot(122)  plt.hist(equ.flatten(), 256)plt.plot(cdf_normalized_eq, color='r')plt.show()cv2.imshow('original', o)cv2.imshow('gray_equ', equ)cv2.waitKey()cv2.destroyAllWindows()cv2.waitKey(1)

### Color 均衡化處理實例

In [ ]:
import cv2import numpy as npimg = cv2.imread('../assets/images/basic/lenaColor.png')gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)equ = cv2.equalizeHist(gray)     # 灰度圖均衡化result1 = np.hstack((gray, equ))  # 水平拼接原圖和均衡圖cv2.imshow('grey_equ', result1)(b, g, r) = cv2.split(img)        # 彩色影像均衡化,需要分解通道 對每一個通道均衡化bH = cv2.equalizeHist(b)gH = cv2.equalizeHist(g)rH = cv2.equalizeHist(r)equ2 = cv2.merge((bH, gH, rH))    # 合併每一個通道result2 = np.hstack((img, equ2)) # 水平拼接原圖和均衡圖cv2.imshow('bgr_equ', result2)cv2.waitKey()cv2.destroyAllWindows()cv2.waitKey(1)

### 自適應直方圖均衡 Adaptive Histogram Equalization AHE> 為了提高影像的`區域性對比度`, 將影像分成`若干子塊對子塊進行HE處理`，這便是AHE（自適應直方圖均衡化）它在每一個小區域內`（預設 8×8 ）`進行直方圖均衡化。當然，如果有噪點的話，噪點會被放大，需要對小區域內的對比度進行了限制。> * 演算法 ：與一般的直方圖均衡 (全局) 相比，AHE 透過計算圖像每一個顯著`區域的直方圖`，重新分佈圖像的亮度值來改變影像對比度。> * 優點 ：適合於改善影像的區域性對比度以及獲得更多的影像細節。> * 缺點 ：在對比度增強的同時，也放大了影像的噪音，>限制對比度自適應直方圖均衡 Contrast Limited Adaptive Histogram Equalization CLAHE 演算法 ：CLAHE與AHE都是局部均衡化，也就是把整個圖像分成許多小塊Tiles (OpenCV default為8×8)，對每個小塊進行均衡化。這種方法主要對於圖像直方圖不是那麼單一的圖像(e.g. 多峰情況)比較實用。所以在每一個的區域中，直方圖會集中在某一個小的區域中> * clipLimit 參數表示對比度的大小。 > * tileGridSize 參數表示每次處理塊的大小 。<br>https://iter01.com/518513.html

In [ ]:
import cv2import numpy as npimg = cv2.imread('../assets/images/basic/lenaColor.png', 0)# 全域性直方圖均衡equ = cv2.equalizeHist(img)# 自適應直方圖均衡# @param clipLimit Threshold for contrast limiting.clahe = cv2.createCLAHE(clipLimit = 100.0, tileGridSize = (8, 8))  # AHE   # default cliplimit  = 40 cl1 = clahe.apply(img)# 水平拼接三張影像result1 = np.hstack((img, equ, cl1))cv2.imshow('clahe_result', cv2.resize(result1, (0,0), fx=.8, fy=.8))fig=plt.figure(figsize=(12, 8))plt.subplot(221)plt.hist(img.flatten(), 256, [0, 255], label='original image'), plt.legend()plt.subplot(222)plt.hist(equ.flatten(), 256, [0, 255], label='equalize image', color='orange'), plt.legend()plt.subplot(223)plt.hist(cl1.flatten(), 256, [0, 255], label='clahe image', color='g'), plt.legend()plt.subplot(224)plt.hist(img.flatten(), 256, [0, 255], label='original image')plt.hist(equ.flatten(), 256, [0, 255], label='equalize image')plt.hist(cl1.flatten(), 256, [0, 255], label='clahe image')plt.legend()cv2.waitKey()cv2.destroyAllWindows()

---

## 🎯 實作練習

### 練習 1: 比較不同影像的直方圖

載入多張不同亮度的影像，計算並比較其直方圖分布。

### 練習 2: 自訂灰階映射

實作自訂的灰階映射函數，將影像的特定亮度範圍拉伸到完整的 0-255 範圍。

### 練習 3: 區域對比度增強

使用 CLAHE 對不同 `clipLimit` 和 `tileGridSize` 參數進行實驗，觀察效果差異。

---

## 📝 本章總結

### 核心概念回顧

1. **直方圖 (Histogram)**: 反映影像像素強度 0-255 的分佈情形
2. **cv2.calcHist()**: 計算影像直方圖的核心函數
3. **直方圖均衡化 (HE)**: 拉伸像素強度分佈範圍，增強全域對比度
4. **CLAHE**: 限制對比度的自適應直方圖均衡，處理區域對比度
5. **灰階拉伸**: 針對特定亮度範圍進行線性映射

### 應用場景

- 📸 **攝影增強**: 修正過曝或欠曝照片
- 🏥 **醫學影像**: 增強 X 光片、CT 掃描的可視性
- 🛰️ **遙測影像**: 改善衛星影像對比度
- 🔍 **特徵提取**: 預處理步驟，提升特徵偵測效果

### 下一步學習

- ➡️ [3.2.2 影像濾波器](3.2.2_image_filtering.ipynb)
- ➡️ [3.2.3 邊緣偵測](3.2.3_edge_detection.ipynb)
- ➡️ [3.2.4 形態學操作](3.2.4_morphological_operations.ipynb)

---

**模組完成日期**: 2025-10-13  
**維護者**: OpenCV Tools Team  
**版本**: 1.0.0